# Importing the data

Originally, the file I imported is the ```ign.csv``` file from [here](https://www.kaggle.com/egrinstein/20-years-of-games). The section below demonstrates how I scraped the IGN website to find reviews to add in more data.

In [1]:
import pandas as pd

In [6]:
df = pd.read_csv('ign.csv')

# Scraping reviews to add in more data

It was kind of the compiler of this data to obtain the URLs. It made scraping IGN for its reviews a lot easier! I'm using BeautifulSoup to scrape the reviews. Overall, this worked for a good number of entries. A couple possibilities for why some reviews are not in the data:

1. Some games have video reviews, and I'm not really at a point where I would like to try to pull data from a video.

2. Maybe a review was not in the ```top-mixed-media``` class of ```div``` tags.

I noticed that some pages have multiple reviews, but to simplify the data-gathering a bit, I wanted to pick one of them. Any one would do.

The ```urllib``` imports below are strictly ```Python 3.x```. For ```Python 2.x``` the module should be ```urllib2```. I imported the ```time.sleep``` method so that IGN doesn't get mad at me for inundating them with HTTP requests.

In [7]:
from urllib.request import urlopen
from urllib.error import HTTPError
from time import sleep

from bs4 import BeautifulSoup

I've found that sometimes reviews pages do not exist for some reason, either because the link was not found in the HTML, or if it did, the page was missing for some reason. In either case, I handled that by simply returning an empty string for the review.

In [8]:
def get_review(row):
    game_link = row['url']
    game_address = 'http://www.ign.com' + game_link
    
    try:
        main_game_page = urlopen(game_address)
        main_page_soup = BeautifulSoup(main_game_page, 'html.parser')

        likely_rev_location = main_page_soup.find('div', attrs={'class': 'top-mixed-media'})

        links = likely_rev_location.find_all('a')

        link_to_review = None

        for link in links:
            if 'review' in link.get('href').lower():
                link_to_review = link.get('href')

        review_text = ''

        if link_to_review:
            sleep(1.25)
            rev_page = urlopen(link_to_review)
            rev_page_soup = BeautifulSoup(rev_page, 'html.parser')

            review_div = rev_page_soup.find('div', attrs={'class': 'article-content'})
            if review_div:
                review_text = ' '.join(review_div.get_text().replace('\n','').replace('Share.','').split())
            else:
                review_text = ''
    except HTTPError:
        review_text = ''
        
    return review_text

In [9]:
df['review'] = df.apply(get_review, axis=1)

I'd rather not rescrape this every single time, so I'll save the file.

In [10]:
df.to_csv('ign_with_reviews.csv', sep=',', index=False)

In [2]:
df = pd.read_csv('ign_with_reviews.csv').drop(columns=['score_phrase','url']).fillna('')

# Removing duplicates

Some games (e.g. Overwatch, The Legend of Zelda: Twilight Princess) are on multiple platforms. For a recommender system that is based on terms, it's very easy for cosine similarity methods to make the trivial match because the titles are exactly the same. I'll get around this by combining entries on different platforms and simply list the platforms next to each other in the ```platform``` field (e.g. ```'PC XBox Wii'```).

It's conceivable that there are different reviews for the games on different platforms (e.g. Twilight Princess on Wii vs. GameCube, or Dark Souls on the PlayStation vs. PC), but even if there are issues with differences between platforms, it would affect only a small subset of games that I wouldn't even worry about it.

In [3]:
titles = df['title'].unique()

for title in titles:
    platforms = df[df['title'] == title]['platform'].unique()
    platforms_string = ' '.join(platforms).strip()
    
    same_title_indices = df[df['title'] == title].index.values
    
    if len(same_title_indices) > 1:
        try:
            df['platform'].iloc[same_title_indices[0]] = platforms_string
            df = df.drop(same_title_indices[1:]).reset_index(drop=True)
        except IndexError:
            print('IndexError at: {}'.format(same_title_indices))
    

c:\users\jeff\documents\data_sci\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


And to create a version without the duplicates

In [13]:
df.to_csv('ign_with_reviews_no_dup.csv', sep=',', index=False)

In [7]:
df = pd.read_csv('ign_with_reviews_no_dup.csv').fillna('')

# Examining the corpus

To gain some general insights, it would be good to observe what words pop up a lot in reviews for great games. Here, I'll arbitrarily say that great games are ones with scores of 8.5 or more.

First I will combine all the relevant fields together.

In [8]:
def combine_fields(row):
    return (''.join(row['title']) + ' ' + ''.join(row['platform']) + ' ' + ''.join(row['genre']) + ' ' + ''.join(row['review']))

df['combined'] = df.apply(combine_fields, axis=1)

Next, I'm going to use a simple count vectorizer to see how many times words come up.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

Before I start, I'm going to introduce some stop words from [NLTK](https://www.nltk.org).

In [10]:
from nltk.corpus import stopwords

In [97]:
stop_words = set(stopwords.words('english'))

In [90]:
# it looks like some HTMl got scraped up accidentally
stop_words.update(['data','div','span','class','objectcard','widget','autolink',
                   'hover','objectrating','objectlegacyid','ratingsubtext','objecttype',
                  'objectboxart','url','ul','li','href','none','mediatype','displayprice','media','image',
                  'display','autocommerce','commerce', 'zd', 'sub', 'objectreviewscore','img','src'])

stop_words.update([''])

Eventually, I'd like to add to this set of ```stop_words``` by examining the vocabulary of the more frequent words. Even without the typical stop words set by NLTK, the threshold for document frequency has to be moved down a lot! I eventually found that a maximum document frequency of about 0.3 gave words that could be interesting. Anything higher, and I see words that were unintentionally scraped because of HTML that leaked into the review text.

In [164]:
count_vectorizer = CountVectorizer(stop_words=stop_words, min_df = 0.05, max_df = 0.35, ngram_range=(1,3))

counts = count_vectorizer.fit_transform(df[df['score'] >= 8.5]['combined'])

words = count_vectorizer.get_feature_names()

counts_matrix = counts.toarray()
counts_list = counts_matrix.sum(axis=0)

counts_df = pd.DataFrame()

counts_df['words'] = words
counts_df['counts'] = counts_list

In [165]:
sorted_counts = counts_df.sort_values(by=['counts'], ascending=False).reset_index().drop(columns=["index"])

sorted_counts.index = sorted_counts['words'].drop(columns=['words'])

In [166]:
%matplotlib
sorted_counts.plot(kind="barh", use_index=True)

Using matplotlib backend: TkAgg


With this plot, it's quite easy to gain some insights as to what makes a good game. I had to do a bit of manual scanning for some parts, though in principle if one knew all the possible words to look for, this could be automated.

# What platforms are mentioned the most in these reviews?

*Based on this cut*, here is what seems to be the platforms that are the most mentioned:

1) PC

2) XBox

3) Wii

4) XBox 360

6) Game Boy

7) PlayStation

8) GameCube

To be fair to the analysis, single characters like "1" or "a" tend to be filtered out, so it's hard to say *which* of the PlayStations item number 7 refers to, and which Wii number 3 refers to. What's surprising to me, personally, was that the PlayStation platform was less popular than the Game Boy.

I'm not too surprised that the PC has a lot of mentions, as the personal computer has been around for *decades*. The only parts that change are the internals, and ultimately the question of where exactly one should place the PC on this list is exactly the ship of Theseus question of philosophy. Being able to change out parts and improve graphics and performance is likely a major reason why it is at the top of the list. While games, of course, look great on a console, typically the same game on a PC can look and perform so much better (see Fallout and Skyrim, for example).

Nintendo is a company and not a platform, but it should be mentioned that the word Nintendo was detected between XBox and Wii (in terms of word count). This is a company that has been around forever and knows how to make great games, so it's no surprise at all that their number of mentions is way up there.

# What types of games are mentioned the most in these reviews?

I could imagine that some of these words could actually come from a combination of the game genre and the review. As such, I won't necessarily restrict the question to the genre of games, but rather to the types of games, which could extend beyond something like "RPG" or "Fantasy" and so on.

1) RPG

2) Strategy

3) Racing

4) Puzzle

5) Online

6) Multiplayer

7) Sports

8) Fighting

9) Portable

10) Platformer

11) Single Player

12) Action adventure

Something interesting here is that multiplayer is mentioned more than single player (detected by setting the CountVectorizer to spot bigrams). The difference is not too large, but the ranking seems interesting enough to note that perhaps the social aspect of games is a huge appeal.

IGN reviewers seem to love talking about RPGs, online games, multiplayer games, and sports games, but I do omit a few of the top listings because I think that some of these types of games could be confused with mechanisms in the game. Strategy can be a genre of game, but the word itself could appear in a review for an RPG, or a racing game, or even some online or multiplayer game. It's not entirely clear just based on word count alone what that means, but IGN reviewers seem to like games that have strategy in it. Racing games like Mario Kart, F-Zero, Gran Turismo, etc. indeed form their own genre, but similar to strategy, I think that the term racing could actually be confused when combined with other genres. In RPGs, you might have racing mini-games; in puzzles you might be racing against a clock. There are so many things that racing could actually appear in, so it's not completely clear how much of racing refers to the actual genre vs. the act of racing against something in-game. 

# Building a recommender system

I'll be comparing the use of a vectorizer using absolute term frequency to one using term frequency-inverse document frequency.

The routine for cosine similarity and retrieving recommendations was written with a lot of help from a DataCamp tutorial seen [here](https://www.datacamp.com/community/tutorials/recommender-systems-python). All the data processing, web scraping, and model adjusting is mine, though!

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity

This particular recommender system should never recommend terrible games. To be a bit of an elitist here (and to get around memory issues), I will only recommend games that have a score of 8.5 and above.

In [81]:
df_rec = df[df['score'] >= 8.5]

Some functions to aid in combining the metadata and the review into a huge body of text.

Here's a helper function to return the 10 most cosine-similar games given the game title, platform, genre, and review. This, of course, assumes that the title passed into ```get_recommendations``` exists.

In [63]:
df_rec = df_rec.reset_index()
indices = pd.Series(df_rec.index, index=df_rec['title'])

def get_recommendations(title, cosine_similarity):
    if indices[title].shape is not ():
        index = indices[title].iloc[0]
    else:
        index = indices[title]
    scores = list(enumerate(cosine_similarity[index]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    scores = scores[1:11]
    
    rec_indices = [i[0] for i in scores]
    
    return df_rec[['title','genre','platform']].iloc[rec_indices]

### Combining all fields together

In [110]:
extra_stopwords = {'ex', 'revolution', 'throne', 'frozen', 'war', 'iii'}

tfidf = TfidfVectorizer(stop_words=stop_words.union(extra_stopwords), min_df = 0, max_df = 0.04)
tfidf_matrix = tfidf.fit_transform(df_rec['combined'])

cos_sim_tfidf = cosine_similarity(tfidf_matrix, tfidf_matrix)

### Deus Ex recommendations

The only thing I find really off is Nintendogs: Lab & Friends and Cooking Dash 3: Thrills & Spills.

In [111]:
print("I think you might like:")
get_recommendations('Deus Ex: Mankind Divided', cos_sim_tfidf)

I think you might like:


,title,genre,platform
10934,Deus Ex: Human Revolution,Shooter,PC PlayStation 3 Xbox 360
1819,Deus Ex,RPG,PC Macintosh
4034,Deus Ex: Invisible War,"Action, Adventure",PC Xbox
11915,The Walking Dead: Season Two -- Episode 2: A H...,Adventure,Xbox 360 iPhone PC PlayStation Vita PlayStation 3
106,Dishonored,Action,Xbox 360 PC PlayStation 3
5146,Nintendogs: Lab & Friends,Virtual Pet,Nintendo DS
11660,Stealth Inc.: A Clone in the Dark,Platformer,PlayStation 3 PlayStation Vita
3544,.hack//INFECTION (Part 1),"Action, RPG",PlayStation 2
9,Mark of the Ninja,"Action, Adventure",Xbox 360 PC
10457,Cooking Dash 3: Thrills & Spills,Strategy,PC


### Warcraft 3: The Frozen Throne recommendations

I expected that the word "Warcraft" ought to give matches to other games with the same name. However, this is something that could use some work. Including "Warcraft" into a set of stop words immediately results in some wacky recommendations that likely would not be recommended by any reasonable measure.

In [112]:
print("I think you might like:")
get_recommendations('Warcraft III: The Frozen Throne', cos_sim_tfidf)

I think you might like:


,title,genre,platform
4783,World of Warcraft,RPG,PC
10568,World of Warcraft: Cataclysm,RPG,PC
3131,Warcraft III: Reign of Chaos,Strategy,PC Macintosh
22,World of Warcraft: Mists of Pandaria,RPG,PC
6457,The Sims 2: Seasons,Simulation,PC
12131,World of Warcraft: Warlords of Draenor,RPG,PC
8437,World of Warcraft: Wrath of the Lich King,RPG,PC
8427,World of Warcraft: Wrath of the Lich King (Col...,RPG,PC
11928,Hearthstone: Heroes of WarCraft,"Card, Battle",Macintosh iPad PC
12473,World of Warcraft: Legion,RPG,PC
